##### Copyright 2019 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Image segmentation

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/images/segmentation">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/segmentation.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/images/segmentation.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/images/segmentation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Tutorial ini fokus membahas segmentasi gambar menggunakan <a href="https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/" class="external">U-Net</a> yang telah dimodifikasi.

## Apa itu "image segmentation" ?
Sejauh ini Anda telah mempelajari klasifikasi gambar, di mana tugas neural networks adalah untuk menetapkan label atau kelas dari gambar input. Namun, jika anda ingin tahu di mana suatu objek berada pada gambar, bentuknya seperti apa, piksel mana yang menjadi bagian objek tesebut, dll. Dalam hal ini Anda ingin mengelompokkan pixel gambar, yaitu, setiap piksel objek diberi label. Dengan demikian, tugas segmentasi gambar adalah untuk melatih jaringan saraf untuk menghasilkan "mask" pixel dari gambar. Teknik ini membantu menginterpretasikan gambar pada level yang jauh lebih rendah, mis., Level piksel. Segmentasi gambar memiliki banyak aplikasi dalam dunia medis, self-driving car dan pencitraan satelit.

Dataset yang akan digunkan pada tutorial ini adalah [Oxford-IIIT Pet Dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/), di-develop oleh Parkhi *et al*. Dataset terdiri dari gambar input, dan label masknya. Mask pada label dilabeli pada tiap pixel, sehingga tiap pixel mempunyai 3 kategori:

*   Class 1 : Pixel untuk objek(binatang peliharaan)
*   Class 2 : Pixel untuk border objek
*   Class 3 : pixel background

In [ ]:
!pip install git+https://github.com/tensorflow/examples.git
!pip install tensorflow_datasets

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

from tensorflow_examples.models.pix2pix import pix2pix

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from IPython.display import clear_output
import matplotlib.pyplot as plt

## Download dataset Oxford-IIIT Pets

Dataset yang digunakan sudah ter-include di *tensorflow datasets*, kita tinggal men-downloadnya saja. Khusus untuk dataset segmentasi, tersedia di versi 3+.

In [ ]:
dataset, info = tfds.load('oxford_iiit_pet:3.*.*', with_info=True)

Code di bawah ini adalah contoh sederhana untuk melakukan augmentasi dataset (flipping). Selanjutnya, gambar di normalisasi [0,1]. Selanjutnya, seperti yang disebutkan di atas, tiap pixel dari "mask" segmentasi dilabeli {1, 2, 3}. Untuk memudahkan, kita akan substrcat labelnya dengan 1 menjadi {0, 1, 2}.

In [ ]:
def normalize(input_image, input_mask):
  input_image = tf.cast(input_image, tf.float32) / 255.0
  input_mask -= 1
  return input_image, input_mask

In [ ]:
@tf.function
def load_image_train(datapoint):
  input_image = tf.image.resize(datapoint['image'], (128, 128))
  input_mask = tf.image.resize(datapoint['segmentation_mask'], (128, 128))

  if tf.random.uniform(()) > 0.5:
    input_image = tf.image.flip_left_right(input_image)
    input_mask = tf.image.flip_left_right(input_mask)

  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

In [ ]:
def load_image_test(datapoint):
  input_image = tf.image.resize(datapoint['image'], (128, 128))
  input_mask = tf.image.resize(datapoint['segmentation_mask'], (128, 128))

  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

Dataset yang digunakan telah dibagi menjadi test dan training, so let's continue.

In [ ]:
TRAIN_LENGTH = info.splits['train'].num_examples
BATCH_SIZE = 64
BUFFER_SIZE = 1000
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

In [ ]:
train = dataset['train'].map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test = dataset['test'].map(load_image_test)

In [ ]:
train_dataset = train.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset = test.batch(BATCH_SIZE)

Mari kita lihat contoh gambar input dan labelnya dari dataset.

In [ ]:
def display(display_list):
  plt.figure(figsize=(15, 15))

  title = ['Input Image', 'True Mask', 'Predicted Mask']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
    plt.axis('off')
  plt.show()

In [ ]:
for image, mask in train.take(1):
  sample_image, sample_mask = image, mask
display([sample_image, sample_mask])

## Pendefinisian Model

Model yang digunakan di sini adalah modifikasi U-Net. U-Net terdiri dari encoder(downsampler) dan decoder (upsampler). Agar model ini mempelajari fitur yang robust, dan mengurangi jumlah parameter yang perlu di training, kita gunakan pretrained model sebagai encoder. Jadi encoder yang akan kita gunakan disini adalah pretrained MobileNetV2 model, yang intermediat outputnya dipakai lagi sebgai. Sedang decoder yang akan digunakan adalah upsample block, sama seperti yang diimplemetasikan pada contoh ini [Pix2pix tutorial](https://github.com/tensorflow/examples/blob/master/tensorflow_examples/models/pix2pix/pix2pix.py).

Alasan mengapa ada 3 channel output yang dihasilkan, karena kemungkinan kelas prediksinya adalah 3 untuk tiap pixel. Anggap task ini sebagai multi-classification dimana tiap pixel akan dikalsifikasikan menjadi 3 kelas.

In [ ]:
OUTPUT_CHANNELS = 3

Seperti yang dibahas di atas, encoder yang digunakan adalah MobilenetV2, model telah disiapkan dan tersedia di [tf.keras.applications](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/applications). Encoder juga menghasilkan beberapa output dari intermediate layer (output ini akan digunakan sebagai input dari decoder). Sebagai catatan, encoder tak ikut ditraining pada task kali ini (hanya decoder yang ditraining).

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=[128, 128, 3], include_top=False)

# Use the activations of these layers
layer_names = [
    'block_1_expand_relu',   # 64x64
    'block_3_expand_relu',   # 32x32
    'block_6_expand_relu',   # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project',      # 4x4
]
layers = [base_model.get_layer(name).output for name in layer_names]

# Create the feature extraction model
down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)

down_stack.trainable = False

Decoder/upsampler adalah sebuah series block upsample dari neural network yang tersedia di "tensorflow_examples" (pada task kali ini kita menggunakan pix2pix)

In [ ]:
up_stack = [
    pix2pix.upsample(512, 3),  # 4x4 -> 8x8
    pix2pix.upsample(256, 3),  # 8x8 -> 16x16
    pix2pix.upsample(128, 3),  # 16x16 -> 32x32
    pix2pix.upsample(64, 3),   # 32x32 -> 64x64
]

In [ ]:
def unet_model(output_channels):

  # This is the last layer of the model
  last = tf.keras.layers.Conv2DTranspose(
      output_channels, 3, strides=2,
      padding='same', activation='softmax')  #64x64 -> 128x128

  inputs = tf.keras.layers.Input(shape=[128, 128, 3])
  x = inputs

  # Downsampling through the model
  skips = down_stack(x)
  x = skips[-1]
  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    concat = tf.keras.layers.Concatenate()
    x = concat([x, skip])

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

## Training model
Sekarang, kita tinggal meng-compile dan men-training model. Loss yang akan kita gunakan adalah "sparse categorical crossentropy". Alasan penggunaan fungsi loss ini adalah network kita akan mencoba memprediksi label untuk tiap pixel (sparse) seperti pada multiclass prediction biasa. Pada label segmentasi (target), tiap pixel mempunyai kelas salah satu dari {0, 1, 2}. Network yang telah dibagun mempunyai output 3 channel. Maksudnya tiap channel akan memprediksi sebuah kelas, dan "sparse categorical crossentropy" adalah loss yang cocok untuk skenario ini. Label yang dipilih untuk sebuah pixel adalah channel dengan output terbesar dari yang lain.

In [ ]:
model = unet_model(OUTPUT_CHANNELS)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Sekarang kita lihat arsitektur final dari model yang kita but.

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

Sekarang kita coba lihat output dari model sebelum di training

In [ ]:
def create_mask(pred_mask):
  pred_mask = tf.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

In [ ]:
def show_predictions(dataset=None, num=1):
  if dataset:
    for image, mask in dataset.take(num):
      pred_mask = model.predict(image)
      display([image[0], mask[0], create_mask(pred_mask)])
  else:
    display([sample_image, sample_mask,
             create_mask(model.predict(sample_image[tf.newaxis, ...]))])

In [ ]:
show_predictions()

Let's observe how the model improves while it is training. To accomplish this task, a callback function is defined below. 

In [ ]:
class DisplayCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    clear_output(wait=True)
    show_predictions()
    print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

In [ ]:
EPOCHS = 20
VAL_SUBSPLITS = 5
VALIDATION_STEPS = info.splits['test'].num_examples//BATCH_SIZE//VAL_SUBSPLITS

model_history = model.fit(train_dataset, epochs=EPOCHS,
                          steps_per_epoch=STEPS_PER_EPOCH,
                          validation_steps=VALIDATION_STEPS,
                          validation_data=test_dataset,
                          callbacks=[DisplayCallback()])

In [ ]:
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

epochs = range(EPOCHS)

plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'bo', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.ylim([0, 1])
plt.legend()
plt.show()

## Memprediksi dengan model

Sekarang kita coba memprediksi. Untuk mempersingkat waktu, jumlah epoch yang digunakan kecil dulu, tetapi kita bisa menambah jumlah epoch untuk hasil yang lebih akurat. 

In [ ]:
show_predictions(test_dataset, 3)

## Selanjutnya
Sekarang anda telah memahami apa itu segmentasi gambar dan bagaimana cara kerjanya. Anda bisa mencoba tutorial ini dengan beberapa intermediate layer yang berbeda, atau bahkan pretrained model yang berbeda. Anda juga bisa men-challenge diri sendiri dengan mencoba [Carvana](https://www.kaggle.com/c/carvana-image-masking-challenge/overview) image masking yang tersedia di kaggle.

Anda juga bisa mencoba [Tensorflow Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection) untuk object detection dengan dataset kamu sendiri.

